### 测试模型效果代码

可以修改超参数区域的SAVE_MODEL内容。

**注意：务必保证训练的模型超参数与测试的超参数一致**

In [1]:
import numpy as np
from sklearn.metrics import *
import copy
import utils
import time
import os
from keras.models import load_model
from keras.utils import multi_gpu_model
from keras.layers import *
from keras import Model
import keras.backend as K
import tensorflow as tf
from model import Capsule
# 关闭警告信息
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings("ignore")

##########################################################

label_to_ix=np.load('Data/label_to_ix.npy',allow_pickle=True).item()
test_data=np.load('Data/test_data.npy',allow_pickle=True)
word_to_ix=np.load('Data/word_to_ix.npy',allow_pickle=True).item()

Using TensorFlow backend.


超参数区域，谨慎修改。

set_seed所设置的是随机种子，用于确保效果的可复现性。

In [2]:
# 超参数区域
BATCH_SIZE = 16
SAVE_MODEL = 'Model_save/best_model_capsule_label-0.6753.h5' # 需要测试的模型名
LABEL_NUM = len(label_to_ix.keys())

def set_seed(THATSEED):
    np.random.seed(THATSEED)
    tf.compat.v1.set_random_seed(THATSEED)
set_seed(1)

In [3]:
batch_test = utils.preprocessing(test_data,label_to_ix,BATCH_SIZE,word_to_ix)
label_list =  np.expand_dims(np.arange(LABEL_NUM), axis=0)
label_list = np.repeat(label_list,BATCH_SIZE,axis=0)
# 测试过程，SAVE_MODEL为需要测试效果的模型名字
print("*"*20,"test","*"*20)
model = load_model(SAVE_MODEL,custom_objects={'Capsule':Capsule})

In [27]:
test_y_pred = []
test_y = []
recall = []
losses = []
topk = 10
for batch_data in batch_test:
    batch_y_pred = model.predict([batch_data[0],label_list])
    losses.append(model.evaluate([batch_data[0],label_list], batch_data[2],verbose=0))
    batch_y = batch_data[2]
    test_y_pred.append(batch_y_pred)
    test_y.append(batch_y)
    
    for iii in range(0,len(batch_y_pred)):
                temp={}
                for iiii in range(0,len(batch_y_pred[iii])):
                    temp[iiii]=batch_y_pred[iii][iiii]
                temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
                thistop=int(np.sum(batch_y[iii]))
                hit=0.0
                for ii in temp1[0:max(thistop,topk)]:
                    if batch_y[iii][ii[0]]==1.0:
                        hit=hit+1
                if thistop!=0:
                    recall.append(hit/thistop)

test_y_raw = np.concatenate(test_y_pred,axis=0)
test_y = np.concatenate(test_y,axis=0)
test_y_pred = np.array([[1 if i >= 0.5 else 0 for i in t] for t in test_y_raw])

In [30]:
import eval
names = ['auc_macro','auc_micro']
all_metrics = eval.all_metrics(test_y_pred, test_y, k=10, yhat_raw=test_y_raw, calc_auc=True)
for name in names:
    print("%s: %.4f" % (name,all_metrics[name]))
val_loss,val_acc = np.mean(losses, axis=0)
print("top 10 recall: %.4f" % (np.mean(recall)))
print("test loss: %.4f" % (val_loss))
eval.sk_micro_f1(test_y_pred,test_y)
eval.sk_macro_f1(test_y_pred,test_y)

auc_macro: 0.9084
auc_micro: 0.9837
top 10 recall: 0.8226
test loss: 0.0291
micro precision: 0.7245, micro recall: 0.6324, micro f1: 0.6753.
macro precision: 0.3213, macro recall: 0.2487, macro f1: 0.2645.


0.2644913699155241